### Prepare space in flash

In [1]:
import os, subprocess

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220518_P186_laqueus_DiI_L-A2/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "ants")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Prepare mask

In [4]:
import nrrd
from probemapper.io import save_nifti, load_nifti
import numpy as np

In [5]:
raw = load_nifti(os.path.join(rootdir, "analysis/536BP_10um.nii"))
mask, _ = nrrd.read(os.path.join(rootdir, "analysis/slicer_536BP/Segmentation.seg.nrrd"))
mask = np.swapaxes(mask, 0, 2)
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "536BP_10um_masked.nii"), 10, 10, 10)

In [6]:
raw = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um.nii")
mask = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um_mask.nii.gz")
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10, 10, 10)

### Registration

In [7]:
from probemapper.registration import ANTSParams, ANTSStage, ANTSRegisteration

In [8]:
from probemapper.registration.helper import define_initial_transformation
import numpy as np

aff = np.array([[ 1.0, 0, 0],
                [ 0, 1.0, 0],
                [ 0, 0, 1.0]], dtype='float')
transl = np.array([0, 0, 0], dtype='float')
define_initial_transformation(aff, transl, os.path.join(tmpdir, "initial.mat"))

.mat file saved as /flash/ReiterU/tmp/P186/ants/initial.mat


In [9]:
P = ANTSParams(
    use_float=False,
    interpolation="Linear",
    histogram_matching=False,
    winsorize=[0.05, 1.0],
    init_transform_fx_to_mv=os.path.join(tmpdir, "initial.mat"),
    stages=[
        ANTSStage(
            transform="Affine",
            transform_args="0.1",
            metrics="MI",
            metrics_args="1,128,Regular,0.5",
            convergence="1000x1000x1000,1e-5,15",
            shrink_factors="8x4x2",
            smoothing_sigmas="3x2x1vox"
        ),
        ANTSStage(
            transform="SyN",
            transform_args="0.1,4.0,0.2",
            metrics="CC",
            metrics_args="1,4",
            convergence="500x500x100x30,1e-6,10",
            shrink_factors="8x4x2x1",
            smoothing_sigmas="3x2x1x0vox"
        )
    ]
)

In [10]:
R = ANTSRegisteration(P, ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin", num_threads=12)
R.set_fixed_image(os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10)
R.set_moving_image(os.path.join(tmpdir, "536BP_10um_masked.nii"), 10)
R.set_outputs(tmpdir)

In [11]:
R.run()

########################################
['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsRegistration', '--dimensionality', '3', '--initial-moving-transform', '/flash/ReiterU/tmp/P186/ants/initial.mat', '--interpolation', 'Linear', '--use-histogram-matching', '0', '--float', '0', '--verbose', '1', '--output', '[/flash/ReiterU/tmp/P186/ants/F2M_,/flash/ReiterU/tmp/P186/ants/Warped.nii.gz,/flash/ReiterU/tmp/P186/ants/InvWarped.nii.gz]', '--transform', 'Affine[0.1]', '--metric', 'MI[/flash/ReiterU/tmp/P186/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P186/ants/536BP_10um_masked.nii,1,128,Regular,0.5]', '--convergence', '[1000x1000x1000,1e-5,15]', '--shrink-factors', '8x4x2', '--smoothing-sigmas', '3x2x1vox', '--transform', 'SyN[0.1,4.0,0.2]', '--metric', 'CC[/flash/ReiterU/tmp/P186/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P186/ants/536BP_10um_masked.nii,1,4]', '--convergence', '[500x500x100x30,1e-6,10]', '--shrink-factors', '8x4x2x1', '--smoothing-sigmas', '3x2x1x0vox']


 2DIAGNOSTIC,     6, -6.745482640415e-02, 1.797693134862e+308, 2.5841e+00, 4.2047e-02,
 2DIAGNOSTIC,     7, -6.868879710896e-02, 1.797693134862e+308, 2.6265e+00, 4.2380e-02,
 2DIAGNOSTIC,     8, -6.971637298533e-02, 1.797693134862e+308, 2.6679e+00, 4.1401e-02,
 2DIAGNOSTIC,     9, -7.190862541131e-02, 1.797693134862e+308, 2.7091e+00, 4.1256e-02,
 2DIAGNOSTIC,    10, -7.949195906870e-02, 1.797693134862e+308, 2.7503e+00, 4.1186e-02,
 2DIAGNOSTIC,    11, -8.448137359408e-02, 1.797693134862e+308, 2.7918e+00, 4.1496e-02,
 2DIAGNOSTIC,    12, -1.153601323290e-01, 1.797693134862e+308, 2.8338e+00, 4.1986e-02,
 2DIAGNOSTIC,    13, -1.417989853087e-01, 1.797693134862e+308, 2.8811e+00, 4.7278e-02,
 2DIAGNOSTIC,    14, -2.317977410979e-01, 1.797693134862e+308, 2.9441e+00, 6.3029e-02,
 2DIAGNOSTIC,    15, -2.872356334719e-01, 6.031385137749e-02, 2.9939e+00, 4.9815e-02,
 2DIAGNOSTIC,    16, -2.900615460443e-01, 6.842679858436e-02, 3.0389e+00, 4.4993e-02,
 2DIAGNOSTIC,    17, -2.981861595905e-01, 7.2

 2DIAGNOSTIC,    26, -2.844828299625e-01, 2.523024368348e-04, 1.6147e+01, 2.9296e-01,
 2DIAGNOSTIC,    27, -2.845461789596e-01, 2.755726309810e-04, 1.6399e+01, 2.5193e-01,
 2DIAGNOSTIC,    28, -2.847949840884e-01, 2.919519658687e-04, 1.6659e+01, 2.5982e-01,
 2DIAGNOSTIC,    29, -2.854834467712e-01, 3.111012929618e-04, 1.6905e+01, 2.4550e-01,
 2DIAGNOSTIC,    30, -2.862454499485e-01, 3.328491958602e-04, 1.7147e+01, 2.4249e-01,
 2DIAGNOSTIC,    31, -2.876612591766e-01, 3.699355557527e-04, 1.7416e+01, 2.6892e-01,
 2DIAGNOSTIC,    32, -2.893345331447e-01, 4.249999077046e-04, 1.7831e+01, 4.1465e-01,
 2DIAGNOSTIC,    33, -2.903343724935e-01, 4.805633575931e-04, 1.8089e+01, 2.5850e-01,
 2DIAGNOSTIC,    34, -2.964884404093e-01, 6.364054658460e-04, 1.8361e+01, 2.7173e-01,
 2DIAGNOSTIC,    35, -3.006069389806e-01, 8.343596940949e-04, 1.8616e+01, 2.5501e-01,
 2DIAGNOSTIC,    36, -3.030316438342e-01, 1.034309325873e-03, 1.8971e+01, 3.5463e-01,
 2DIAGNOSTIC,    37, -3.038858169768e-01, 1.1982327221

 2DIAGNOSTIC,   122, -3.590808344945e-01, 1.998148966435e-04, 4.5205e+01, 2.7690e-01,
 2DIAGNOSTIC,   123, -3.594615097150e-01, 2.190466964685e-04, 4.5490e+01, 2.8510e-01,
 2DIAGNOSTIC,   124, -3.604361544130e-01, 2.346993977725e-04, 4.5772e+01, 2.8181e-01,
 2DIAGNOSTIC,   125, -3.612242667019e-01, 2.446176044523e-04, 4.6225e+01, 4.5375e-01,
 2DIAGNOSTIC,   126, -3.613342248161e-01, 2.444410771545e-04, 4.6676e+01, 4.5029e-01,
 2DIAGNOSTIC,   127, -3.613795550992e-01, 2.346053691942e-04, 4.6965e+01, 2.8889e-01,
 2DIAGNOSTIC,   128, -3.614408459018e-01, 2.173282852830e-04, 4.7244e+01, 2.7953e-01,
 2DIAGNOSTIC,   129, -3.614714065845e-01, 1.919754995308e-04, 4.7563e+01, 3.1848e-01,
 2DIAGNOSTIC,   130, -3.615846423175e-01, 1.614882871617e-04, 4.7846e+01, 2.8300e-01,
 2DIAGNOSTIC,   131, -3.617188435780e-01, 1.266496158077e-04, 4.8118e+01, 2.7195e-01,
 2DIAGNOSTIC,   132, -3.620002799728e-01, 9.001205689145e-05, 4.8389e+01, 2.7186e-01,
 2DIAGNOSTIC,   133, -3.629799528019e-01, 7.1097236764

 2DIAGNOSTIC,   218, -4.208175726567e-01, 9.781943159628e-05, 7.4841e+01, 2.9327e-01,
 2DIAGNOSTIC,   219, -4.323726245950e-01, 1.635231743597e-04, 7.5099e+01, 2.5806e-01,
 2DIAGNOSTIC,   220, -4.354183795441e-01, 2.280916927323e-04, 7.5402e+01, 3.0279e-01,
 2DIAGNOSTIC,   221, -4.388078898368e-01, 2.905340580101e-04, 7.5669e+01, 2.6693e-01,
 2DIAGNOSTIC,   222, -4.421570508750e-01, 3.488620144740e-04, 7.5974e+01, 3.0502e-01,
 2DIAGNOSTIC,   223, -4.678928143344e-01, 4.556634590943e-04, 7.6276e+01, 3.0215e-01,
 2DIAGNOSTIC,   224, -5.049416298450e-01, 6.312304413148e-04, 7.6606e+01, 3.2944e-01,
 2DIAGNOSTIC,   225, -5.397774978487e-01, 8.594389818382e-04, 7.6892e+01, 2.8601e-01,
 2DIAGNOSTIC,   226, -5.621385467358e-01, 1.098371263091e-03, 7.7188e+01, 2.9632e-01,
 2DIAGNOSTIC,   227, -5.696798028903e-01, 1.303611331182e-03, 7.7481e+01, 2.9282e-01,
 2DIAGNOSTIC,   228, -5.703082323249e-01, 1.452575104730e-03, 7.7738e+01, 2.5676e-01,
 2DIAGNOSTIC,   229, -5.723514448656e-01, 1.5440891554

 1DIAGNOSTIC,    24, -8.710073144518e-01, 4.573355740977e-04, 2.9050e+01, 1.1070e+00,
 1DIAGNOSTIC,    25, -8.720416096141e-01, 4.083541543134e-04, 3.0165e+01, 1.1149e+00,
 1DIAGNOSTIC,    26, -8.730916093177e-01, 3.635873800668e-04, 3.1273e+01, 1.1073e+00,
 1DIAGNOSTIC,    27, -8.739336491374e-01, 3.215023983562e-04, 3.2404e+01, 1.1319e+00,
 1DIAGNOSTIC,    28, -8.747929114081e-01, 2.845467282748e-04, 3.3507e+01, 1.1021e+00,
 1DIAGNOSTIC,    29, -8.754983483430e-01, 2.518701073069e-04, 3.4649e+01, 1.1423e+00,
 1DIAGNOSTIC,    30, -8.762377061161e-01, 2.240397200458e-04, 3.5764e+01, 1.1148e+00,
 1DIAGNOSTIC,    31, -8.768582321423e-01, 1.993659008627e-04, 3.6875e+01, 1.1118e+00,
 1DIAGNOSTIC,    32, -8.775620745959e-01, 1.789566838165e-04, 3.7998e+01, 1.1230e+00,
 1DIAGNOSTIC,    33, -8.781470203852e-01, 1.604837959443e-04, 3.9125e+01, 1.1267e+00,
 1DIAGNOSTIC,    34, -8.787177583053e-01, 1.451263398668e-04, 4.0245e+01, 1.1204e+00,
 1DIAGNOSTIC,    35, -8.793015130278e-01, 1.3215445214

 1DIAGNOSTIC,   120, -8.901287496591e-01, 1.298744064961e-06, 1.3809e+02, 1.0952e+00,
 1DIAGNOSTIC,   121, -8.901431915415e-01, 1.321540796753e-06, 1.3923e+02, 1.1423e+00,
 1DIAGNOSTIC,   122, -8.901610029925e-01, 1.334110936830e-06, 1.4040e+02, 1.1762e+00,
 1DIAGNOSTIC,   123, -8.901427462889e-01, 1.146301164563e-06, 1.4155e+02, 1.1460e+00,
 1DIAGNOSTIC,   124, -8.901875669742e-01, 1.124910807872e-06, 1.4271e+02, 1.1573e+00,
 1DIAGNOSTIC,   125, -8.902048064047e-01, 1.112492297938e-06, 1.4387e+02, 1.1596e+00,
 1DIAGNOSTIC,   126, -8.902118814564e-01, 1.076007946662e-06, 1.4499e+02, 1.1242e+00,
 1DIAGNOSTIC,   127, -8.901922276355e-01, 9.884360784446e-07, 1.4611e+02, 1.1222e+00,
XXDIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST
 1DIAGNOSTIC,     1, -8.914734023814e-01, 1.797693134862e+308, 1.5675e+02, 1.0634e+01,
 1DIAGNOSTIC,     2, -8.926930903179e-01, 1.797693134862e+308, 1.6606e+02, 9.3164e+00,
 1DIAGNOSTIC,     3, -8.940985569747e-01, 1.7976931348

 1DIAGNOSTIC,    88, -9.134161186564e-01, 1.813924966166e-06, 9.4728e+02, 8.3439e+00,
 1DIAGNOSTIC,    89, -9.133770333696e-01, 1.572380251636e-06, 9.5594e+02, 8.6582e+00,
 1DIAGNOSTIC,    90, -9.134028813616e-01, 1.389826469921e-06, 9.6527e+02, 9.3346e+00,
 1DIAGNOSTIC,    91, -9.134283116042e-01, 1.258207405163e-06, 9.7330e+02, 8.0287e+00,
 1DIAGNOSTIC,    92, -9.134526843999e-01, 1.190922968523e-06, 9.8209e+02, 8.7848e+00,
 1DIAGNOSTIC,    93, -9.134743865910e-01, 1.182478932106e-06, 9.9082e+02, 8.7393e+00,
 1DIAGNOSTIC,    94, -9.134942316648e-01, 1.241829796552e-06, 9.9980e+02, 8.9794e+00,
 1DIAGNOSTIC,    95, -9.134552428392e-01, 1.145440506271e-06, 1.0095e+03, 9.6488e+00,
 1DIAGNOSTIC,    96, -9.134782616410e-01, 1.106542887251e-06, 1.0190e+03, 9.5942e+00,
 1DIAGNOSTIC,    97, -9.135057378125e-01, 1.131411525499e-06, 1.0286e+03, 9.5131e+00,
 1DIAGNOSTIC,    98, -9.135274005378e-01, 1.217341582066e-06, 1.0370e+03, 8.4885e+00,
 1DIAGNOSTIC,    99, -9.135510363639e-01, 1.1657778526

 1DIAGNOSTIC,    76, -9.096347393521e-01, 3.506727697757e-06, 5.6826e+03, 6.0226e+01,
 1DIAGNOSTIC,    77, -9.096581878292e-01, 3.427575225234e-06, 5.7443e+03, 6.1642e+01,
 1DIAGNOSTIC,    78, -9.096810331650e-01, 3.274371394661e-06, 5.8040e+03, 5.9728e+01,
 1DIAGNOSTIC,    79, -9.097417117554e-01, 3.209960579426e-06, 5.8658e+03, 6.1765e+01,
 1DIAGNOSTIC,    80, -9.097623400148e-01, 3.061094668728e-06, 5.9239e+03, 5.8183e+01,
 1DIAGNOSTIC,    81, -9.097823813864e-01, 2.995520996146e-06, 5.9814e+03, 5.7429e+01,
 1DIAGNOSTIC,    82, -9.098019126737e-01, 2.862988173737e-06, 6.0392e+03, 5.7862e+01,
 1DIAGNOSTIC,    83, -9.098605135888e-01, 2.813295642541e-06, 6.1008e+03, 6.1594e+01,
 1DIAGNOSTIC,    84, -9.098782807645e-01, 2.681096869604e-06, 6.1618e+03, 6.0998e+01,
 1DIAGNOSTIC,    85, -9.098949071057e-01, 2.624692063634e-06, 6.2226e+03, 6.0779e+01,
 1DIAGNOSTIC,    86, -9.099128509519e-01, 2.508649305767e-06, 6.2827e+03, 6.0059e+01,
 1DIAGNOSTIC,    87, -9.099703639866e-01, 2.4794468061

### Copy files to bucket

In [12]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P186/ants', 'deigo:/bucket/ReiterU/DBS/data/20220518_P186_laqueus_DiI_L-A2/analysis'], returncode=0)